# What does thid notebook do?  
using agglomerative clustering to find posible decision points in all notebooks  

# File mapping 
sklearn | clustering | ---: `./sklearn_dp.txt`

# Agglomerative Clustering

In [ ]:
%matplotlib notebook

In [ ]:
import os
from tqdm import tqdm
import numpy as np
from sklearn import cluster
import scipy.cluster.hierarchy as sch
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

In [ ]:
from importlib import import_module

In [ ]:
from extract_func import process_file

In [ ]:
import json
with open('./temp.out','r') as f:
    func_counter = json.load(f)["func_counter"]
statsmodels_counter = {k: func_counter[k] for k in func_counter if k.startswith('statsmodels.')}

But what is the point to do clustering before choosing representative functions?
Why can't we just sort all the functions and choose short ones? 

In [ ]:
# nb_path = '/projects/bdata/jupyter/target'
nb_path = '/home/gezhang/data/jupyter/target'
notebooks = os.listdir(nb_path)

filter notebooks (which import sklearn)

In [ ]:
statsmodels_notebooks = []
for i, nb in enumerate(notebooks):
    if i%10000==0:
        print(i)
    with open(os.path.join(nb_path, nb),'r') as f:
#         print('--')
        content = f.read()
    tokens = content.split()
    if 'statsmodels' in tokens:
        statsmodels_notebooks.append(nb)
    

# delete loops:
eg  
**sklearn.tree.DecisionTreeClassifier.fit.fit.fit**  
is equal to   
**sklearn.tree.DecisionTreeClassifier.fit**  
  
the only difference is that the first one is called multiple times  

In [ ]:
def delete_loop(func):
    """
    delete continue loops in a function
    simplify **sklearn.tree.DecisionTreeClassifier.fit.fit.fit** 
          to **sklearn.tree.DecisionTreeClassifier.fit**
    """
    tokens = func.split('.')
    new_tokens = []
    for t in tokens:
        if new_tokens == [] or t!=new_tokens[-1]:
            new_tokens.append(t)
    return '.'.join(new_tokens)

In [ ]:
no_loop_statsmodels_funcs = []
for k in statsmodels_counter:
    t = delete_loop(k)
    if t not in no_loop_statsmodels_funcs:
        no_loop_statsmodels_funcs.append(t)
no_loop_statsmodels_funcs

In [ ]:
func2vector = {}
vector_size = len(statsmodels_notebooks)
err_files = []
for i, nb in enumerate(statsmodels_notebooks):
    if i%10000 == 0:
        print('Log: {} notebooks processed'.format(i))
    funcs = []
    try:
        funcs, linenos = process_file(os.path.join(nb_path, nb))
    except Exception as e:
        err_files.append(nb)
    funcs = [func for func in funcs if func in no_loop_statsmodels_funcs]
    for func in funcs:
#         if not func.startswith('statsmodels'):
#             continue
        if func not in func2vector:
            func2vector[func] = np.zeros(vector_size)
        func2vector[func][i] = 1

In [ ]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [ ]:
np.save('statsmodels_cooccur_mat.npy', cooccur_matrix)

In [ ]:
cooccur_matrix = np.load('keras_cooccur_mat.npy')

In [ ]:
a = 1- cooccur_matrix/cooccur_matrix.max()

In [ ]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.4, affinity="precomputed", linkage="average").fit(a)

In [ ]:
clusters = []
n_samples = len(idx2func)
for child in model.children_:
    clusters.append([])
    for branch in child:
        if branch < n_samples:
            clusters[-1].append(idx2func[branch])
        else:
            clusters[-1]+=clusters[branch-n_samples]

In [ ]:
with open('./statsmodels_cluster.json','w') as fout:
    json.dump(clusters,fout, ensure_ascii=False,indent=2)

In [ ]:
with open('./keras_cluster.json','r') as f:
    clusters = json.load(f)

In [ ]:
temp_clusters = [sorted(c) for c in clusters if len(c)<=3]

In [ ]:
not_root = []
sets = []
has_root = []

def find_root_in_cluster(cluster):
    """
    find a representative function in a cluster like:
     ['sklearn.preprocessing.MinMaxScaler',
      'sklearn.preprocessing.MinMaxScaler.fit_transform',
      'sklearn.preprocessing.MinMaxScaler.transform']
    return a list []
    """
    if len(cluster)>3:
        return []
    cluster = sorted(cluster)
    last_root = '*'
    cluster_roots = []
    for f in cluster:
        if not f.startswith(last_root):
            cluster_roots.append(f)
            last_root = f
        elif f not in has_root:
            has_root.append(f)
    cluster_roots = [r for r in cluster_roots if not any([r.startswith(root) for root in roots])]
    return cluster_roots
            

    

In [ ]:
roots = []
for c in tqdm(temp_clusters):
#     print('-'*20)
#     print(c)
    roots+=find_root_in_cluster(c)
#     print(find_root_in_cluster(c))
roots = sorted(set(roots))
# {r: sklearn_counter[r] for r in roots if r.endswith(".predict")}
# [r for r in roots if sklearn_counter[r]>200]
roots

In [ ]:
statsmodels_dp = {r: statsmodels_counter[r] for r in roots if statsmodels_counter[r]>5 }

In [ ]:
for f in statsmodels_dp:
    print(f)

In [ ]:
with open('./statsmodels_dp.txt','w') as fout:
    fout.write('\n'.join(statsmodels_dp))

In [ ]:
with open('./statsmodels_dp.txt','r') as f:
    statsmodels_dp = f.read().split()

In [ ]:
# extract structure from statsmodels_roots
statsmodels_roots = ['.'.join(f.split('.')[:-1]) for f in statsmodels_dp]

In [ ]:
import statsmodels

In [ ]:
roots_counter = {}
for r in statsmodels_roots:
    if r not in roots_counter:
        roots_counter[r]=0
    roots_counter[r]+=1
frequent_roots = {r: roots_counter[r] for r in roots_counter if roots_counter[r]>=5}
for r in frequent_roots:
#     if r=="statsmodels":continue
    try:
        mod = import_module(r)
#         print(mod.__all__)
        try:
            dps = ['{}.{}'.format(r, obj) for obj in mod.__all__] 
        except:
            dps =  ['{}.{}'.format(r, obj) for obj in dir(mod) if not obj.startswith('_')]
#         dps = ['{}.{}'.format(r, obj) for obj in dir(mod) if not obj.startswith('_')]
        statsmodels_dp+=dps
#         print(dps)
    except Exception as e:
        print(e)
statsmodels_dp = sorted(list(set(statsmodels_dp)))

In [ ]:
for f in statsmodels_dp:
    print(f)

In [ ]:
v1 = func2vector["sklearn.cluster.Birch.fit"]
v2 = func2vector["sklearn.cluster.Birch"]
v3 = func2vector["sklearn.cluster.AgglomerativeClustering.labels_.astype"]
np.linalg.norm(v1- v3)